In [1]:
from exp.utils import *
from exp.models import *
from exp.losses import *
from tqdm.notebook import tqdm
from multiprocessing import Pool

import torch
import torch.nn as NN
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

In [2]:
labels = get_labels()
label = labels[0]

In [3]:
# Seed
seed = 92
seed_everything(seed)

# Load data
train_df, valid_df, test_df = get_dataframes(include_labels=labels, 
                                             small=False)

/home/favi/work/crx8/exp/utils.py:179: UserWarning: Train-Val-Split currently with patient overlap!
  warnings.warn("Train-Val-Split currently with patient overlap!")


In [4]:
train_df.head()

,Image Index,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],...,Cardiomegaly,Emphysema,Mass,Pleural_Thickening,Nodule,Atelectasis,Pneumonia,Effusion,Fibrosis,Infiltration
1705,00000459_057.png,30,459,55,F,PA,2698,2845,0.143000,0.143000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87289,00021552_011.png,24,21552,53,M,PA,2021,2021,0.194311,0.194311,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73238,00018029_000.png,0,18029,66,M,PA,2870,2991,0.143000,0.143000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1564,00000416_005.png,5,416,67,F,PA,2992,2991,0.143000,0.143000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67357,00016630_011.png,10,16630,30,M,AP,2500,2048,0.168000,0.168000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
train_df, test_df = get_label_dfs(reduced=False)

In [6]:
train_df.head()

,Image Index,Edema,Consolidation,Pneumothorax,Hernia,Cardiomegaly,Emphysema,Mass,Pleural_Thickening,Nodule,...,Infiltration,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
0,00000001_000.png,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0,1,57,M,PA,2682,2749,0.143,0.143
1,00000001_001.png,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1,1,58,M,PA,2894,2729,0.143,0.143
2,00000001_002.png,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,2,1,58,M,PA,2500,2048,0.168,0.168
3,00000002_000.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,2,80,M,PA,2500,2048,0.171,0.171
12,00000004_000.png,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0,4,82,M,AP,2500,2048,0.168,0.168


In [7]:
def get_train_valid(data, val_size=0.2, seed=92):
    # Currently with patient overlap!
    warnings.warn("Train-Val-Split currently with patient overlap!")
    labels = get_labels()
    X = data[[c for c in data.columns if c not in labels]]
    y = data[labels]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=val_size, random_state=seed)
    return X_train, X_test, y_train, y_test

In [8]:
from collections import Counter

In [9]:
counter = Counter(train_df["Patient ID"].values)

In [10]:
test = counter.most_common()
value = [a for _, a in test]
weight = [1 for _ in test]

In [11]:
val_size = 0.2 * train_df.shape[0]

In [132]:
z = torch.ones((1, 2), requires_grad=True)
a = torch.Tensor([0.1, .6]) + z
b = torch.Tensor([0.1, .6])
c = a + b
c.requires_grad

True

In [122]:
m = NN.Threshold(0.5, 0)
n = NN.Threshold(-0.9999999999999999, 0)
input = torch.Tensor([0.1, .6])
#output = m(input)
input, m(input), m(input) -1 

(tensor([0.1000, 0.6000]),
 tensor([0.0000, 0.6000]),
 tensor([-1.0000, -0.4000]))

In [153]:
class Picker(NN.Module):
    def __init__(self, input_data): 
        super().__init__()
        self.input_data = input_data
        self.inp_shape = input_data.shape
        self.threshold = NN.Threshold(0.5, 0)
    def forward(self, X):
        #z = torch.ones(*self.inp_shape, requires_grad=True)
        #comp = (z + (X > 0.5)).float()
        #ans = torch.matmul(self.input_data, comp.T)
        ans = torch.matmul(self.input_data, self.threshold(X.T))
        return ans
    def get_indices(self, X):
        return torch.where(X > 0.5)[0]

In [178]:
class PickerNet(NN.Module):
    def __init__(self, value, hidden_size):
        super(PickerNet, self).__init__()
        self.l1 = NN.Linear(len(value), hidden_size)
        #self.l2 = NN.Linear(hidden_size, hidden_size)
        self.l3 = NN.Linear(hidden_size, len(value))
        self.r1 = NN.ReLU()
       # self.r2 = NN.ReLU()
        self.r3 = NN.ReLU()
        self.s = NN.Sigmoid()
        self.p = Picker(torch.Tensor(value))
    
    def forward(self, X):
        X = self.r1(self.l1(X))
        #X = self.r2(self.l2(X))
        X = self.r3(self.l3(X))
        X = self.p(self.s(X))
        return X
    
    def get_indices(self, X):
        X = self.r1(self.l1(X))
        #X = self.r2(self.l2(X))
        X = self.r3(self.l3(X))
        X = self.s(X)
        return self.p.get_indices(X)

In [179]:
model = PickerNet(value, 1000)
torch.set_grad_enabled(True) 

In [171]:
model

PickerNet(
  (l1): Linear(in_features=28008, out_features=1000, bias=True)
  (l2): Linear(in_features=1000, out_features=1000, bias=True)
  (l3): Linear(in_features=1000, out_features=28008, bias=True)
  (r1): ReLU()
  (r2): ReLU()
  (r3): ReLU()
  (s): Sigmoid()
  (p): Picker(
    (threshold): Threshold(threshold=0.5, value=0)
  )
)

In [ ]:
#model = NN.Sequential(
#    NN.Linear(len(value), len(value)),
#    NN.ReLU(),
#    NN.Linear(len(value), len(value)),
#    NN.ReLU(),
#    NN.Sigmoid(),
#    Picker(torch.Tensor(value))
#)

In [ ]:
t = torch.Tensor([4, 3, 1, 7])

pred = (torch.rand((2, 4)) > 0.5).float()
print(pred)
t @ pred.T
#pred @ t.T




#self.input_data[indices].sum()
#inds
#a = torch.where(t > 0.5)
#a[0]

In [ ]:
t[a]

In [ ]:
model

In [ ]:
#pred = model(torch.Tensor([value, value]))
pred = model(torch.Tensor(value))

In [ ]:
pred

In [96]:
y_hat = model(X)
y_hat

tensor([17326.6348], grad_fn=<SqueezeBackward3>)

In [ ]:
criterion(torch.Tensor([pred]), torch.Tensor([val_size]))

In [163]:
class PickerLoss:
    def __init__(self, red_size):
        self.mse = NN.MSELoss()
        self.factor = torch.Tensor([red_size])
    def __call__(self, X, y):
        base_loss = torch.sqrt(self.mse(X, y))
        pred_loss = torch.pow(2 * torch.abs(torch.abs(X - 0.5) - 0.5), 2)
        return base_loss + pred_loss / self.factor
        

In [164]:
import copy

In [193]:
def train_it(red_size):
    model = PickerNet(value, 1000)
    losses = []
    device = get_cpu()
    lr = 1e-4
    model = model.to(device)
    model.p.input_data = model.p.input_data.to(device)
    criterion = PickerLoss(red_size=red_size)
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)


    best_model_state = copy.copy(model.state_dict())
    patience = 5
    patience_count = 0
    i = 0
    X = torch.Tensor([value])
    y = torch.Tensor([val_size])
    X, y = X.to(device), y.to(device)
    while True:
        optimizer.zero_grad()
        pred = model(X)
        loss = criterion(pred, y)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()

        if i > 2:
            if losses[-1] >= losses[-2]: 
                best_model_state = copy.copy(model.state_dict())
                if patience_count == patience:
                    patience_count = 0
                    lr /= 3
                    optimizer.param_groups[0]["lr"] = lr
                    #print("LR:", lr)
                else:
                    patience_count += 1
            else: patience_count = 0

        #print(losses[-1])

        if lr < 1e-14: break
        i += 1
    model.load_state_dict(best_model_state)
    inds = model.get_indices(X[0])
    inds = [i for i in inds.detach().numpy().astype(np.uint8)]
    
    
    return np.abs(np.array(value)[inds].sum() - val_size)


In [194]:
red_sizes = [1e-4, 1e-6, 1e-8, 1e-10, 1e-12, 1e-14]
results = []
for red_size in red_sizes:
    print(f"Size: {red_size}")
    results.append({"red_size": red_size, "result": train_it(red_size)})

Size: 0.0001
Using the CPU!


KeyboardInterrupt: 

In [188]:
import math
np.abs(-1)

1

In [180]:
losses = []
device = get_cpu()
lr = 1e-4
model = model.to(device)
model.p.input_data = model.p.input_data.to(device)
criterion = PickerLoss(red_size=sum(value))
optimizer = torch.optim.Adam(model.parameters(),lr=lr)


best_model_state = copy.copy(model.state_dict())
patience = 5
patience_count = 0
i = 0
X = torch.Tensor([value])
y = torch.Tensor([val_size])
X, y = X.to(device), y.to(device)
while True:
    optimizer.zero_grad()
    pred = model(X)
    loss = criterion(pred, y)
    losses.append(loss.item())
    loss.backward()
    optimizer.step()
    
    if i > 2:
        if losses[-1] >= losses[-2]: 
            best_model_state = copy.copy(model.state_dict())
            if patience_count == patience:
                patience_count = 0
                lr /= 3
                optimizer.param_groups[0]["lr"] = lr
                print("LR:", lr)
            else:
                patience_count += 1
        else: patience_count = 0
    
    print(losses[-1])
    
    if lr < 1e-14: break
    i += 1
model.load_state_dict(best_model_state)

Using the CPU!
57176.7265625
55276.5546875
37318.53125
25386.734375
24268.15234375
24122.953125
23916.05859375
23667.240234375
23337.123046875
23236.474609375
23035.564453125
22795.9375
22545.3828125
22343.0625
22144.041015625
21969.92578125
21827.625
21669.9609375
21431.12890625
21275.55078125
21031.955078125
20860.06640625
20676.998046875
20504.35546875
20267.203125
20136.2578125
19932.435546875
19709.5703125
19470.359375
19330.8515625
19187.4375
18970.56640625
18685.23828125
18500.298828125
18311.51171875
18229.05859375
18024.46875
17879.3046875
17783.62109375
17573.44921875
17415.41015625
17298.56640625
17160.11328125
16942.04296875
16800.228515625
16691.05859375
16474.56640625
16305.40234375
16156.7392578125
15846.677734375
15724.314453125
15552.9091796875
15404.4541015625
15180.74609375
14984.197265625
14834.1591796875
14674.0
14552.205078125
14401.8798828125
14210.1611328125
14005.6796875
13827.529296875
13799.3681640625
13766.166015625
13731.5244140625
13698.396484375
13679.936

11908.291015625
11908.265625
11908.0517578125
11907.9560546875
11907.767578125
11907.765625
11907.4873046875
11906.2998046875
11906.2548828125
11906.1669921875
11906.14453125
11906.12109375
11906.119140625
11904.705078125
11904.7041015625
11904.4248046875
11904.404296875
11904.34375
11904.34375
11904.1083984375
11904.0302734375
11903.953125
11903.818359375
11903.72265625
11903.572265625
11903.552734375
11903.533203125
11903.40234375
11903.4013671875
11903.345703125
11903.1796875
11902.6103515625
11902.609375
11902.41796875
11902.400390625
11902.3994140625
11902.3125
11902.294921875
11902.16015625
11902.091796875
11902.0908203125
11902.056640625
11901.9228515625
11901.9052734375
11901.7734375
11901.7734375
11901.755859375
11901.642578125
11901.625
11901.513671875
11901.51171875
11901.259765625
11901.212890625
11901.2119140625
11901.2109375
11901.1337890625
11901.1025390625
11900.99609375
11900.9794921875
11900.9189453125
11900.7109375
11900.7099609375
11900.708984375
11900.693359375
119

<All keys matched successfully>

In [91]:
model.load_state_dict(best_model_state)

<All keys matched successfully>

In [181]:
inds = model.get_indices(X[0])
inds

tensor([    4,     6,    17,  ..., 28002, 28003, 28004])

In [182]:
inds = [i for i in inds.detach().numpy().astype(np.uint8)]

In [183]:
np.array(value)[inds].sum(), val_size

(277220, 17304.8)

In [84]:
np.array(value)[inds].sum() - val_size

399796.2

In [ ]:
X = torch.Tensor([value])
y = torch.Tensor([val_size])
optimizer.zero_grad()
pred = model(X)
loss = criterion(torch.Tensor(pred), y)
#losses.append(loss.item())
optimizer.step()

In [40]:
value[inds].sum()

TypeError: only integer tensors of a single element can be converted to an index

In [ ]:
(pred > 0.5).sum(axis=1).shape

In [ ]:
def knapsack(n, C):
    print(n)
    if store[n, C] != -1: return store[n, C]
    if n == 0 or C == 0: result = 0
    elif weight[n] > C: result = knapsack(n-1, C)
    else:
        tmp_1 = knapsack(n-1, C)
        tmp_2 = value[n] + knapsack(n-1, C-weight[n])
        if tmp_1 > tmp_2 : result = tmp_1
        else: result = tmp_2
    store[n, C] = result
    return result

In [ ]:
n = 25
C = 500
indices = []

store = np.zeros((n+1, C+1)) - 1
knapsack(n, C)

In [ ]:
store

In [ ]:
test

In [ ]:
#pairs = [n for _,n in counter.most_common()]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(ns[:-27500], bins=25);

In [ ]:
val_size = 0.2
n_images = data.shape[0]


In [ ]:
test = [(1, 2), (2, 1), (3, 3), (4, 2), (5, 4)]

sum_it = lambda a: sum([el[1] for el in a])
import math

In [ ]:
val_size = 0.2
n_images = train_df.shape[0]

pairs = sorted(counter.most_common(), key=lambda p: p[1])

In [ ]:
[{"value": i+1, "trace": [i]} for i in range(8)]

In [ ]:
#x = [([1], 1), ([2], 2), ([3], 3), ([4], 4), ([5], 5), ([6], 6), ([7],7), ([8],8)]
x = [{"value": n, "trace": [i]} for i, n in counter.most_common(1000)]
valsize=0.2
sum_it = lambda arr: sum([e["value"] for e in arr])
x_sum = sum_it(x)#sum([y[1] for y in x])
n_x = len(x)
n_goal = math.floor(valsize*n_x)
v_goal = x_sum*valsize

level = 1
lookahead = 1
fov = 1

heuristic = level / n_goal * v_goal

n_goal, v_goal

In [ ]:
n_goal

In [ ]:
def compute_level(x, x_prev, n_goal, v_goal, level=1):
    if x_prev["value"] >= v_goal: return [x_prev], x
    heuristic = level / n_goal * v_goal - x_prev["value"]
    x_level = [{"trace": v["trace"], "value": np.abs(v["value"]-heuristic)} for v in x]
    x_level = sorted(x_level, key=lambda el: el["value"])

    x_best = []
    x_remaining = []
    for idx, el in enumerate(x_level):
            if idx == 0: 
                x_best.append(
                    {"trace": [*x_prev["trace"], *el["trace"]], 
                     "value": el["value"] + x_prev["value"]})
            else:
                x_remaining.append(el)
    return x_best, x_remaining

In [ ]:
for step in range(n_goal):
    if step == 0: best, rem = compute_level(x, {"trace": [], "value": 0}, n_goal, v_goal)
    best, rem = compute_level(rem, best[0], n_goal, v_goal, level=2)

In [ ]:
best

In [ ]:
best, rem = compute_level(x, {"trace": [], "value": 0}, n_goal, v_goal)
best, rem = compute_level(rem, best[0], n_goal, v_goal, level=2)
best, rem

In [ ]:
iterations = range(2, math.ceil(len(test)/2)+1)
totalscore = sum_it(test)
halftotalscore = totalscore/2.0

oldmoves = {}

for p in test:
    people_left = test[:]
    people_left.remove(p)
    oldmoves[p[0]] = people_left

if iterations == []:
    print("Hi")
    solution = min(map(lambda i: (abs(float(i)-halftotalscore), i), oldmoves.keys()))
    assert False
    #return (solution[1], sum(oldmoves[solution[1]]), oldmoves[solution[1]])    

for n in iterations:
    newmoves = {}
    for total, roster in oldmoves.items():
        for p in roster:
            people_left = roster[:]
            people_left.remove(p)
            newtotal = total+p[1]
            if newtotal > halftotalscore: continue
            newmoves[newtotal] = people_left
    oldmoves = newmoves

iterations, totalscore

In [ ]:
solution[1], sum(oldmoves[solution[1]]), oldmoves[solution[1]]

In [ ]:
def team(t):
    iterations = range(2, math.ceil(len(t)/2)+1)

    totalscore = sum(t)
    halftotalscore = totalscore/2.0

    oldmoves = {}

    for p in t:
        people_left = t[:]
        people_left.remove(p)
        oldmoves[p] = people_left

    if iterations == []:
        solution = min(map(lambda i: (abs(float(i)-halftotalscore), i), oldmoves.keys()))
        return (solution[1], sum(oldmoves[solution[1]]), oldmoves[solution[1]])

    for n in iterations:
        newmoves = {}
        for total, roster in oldmoves.items():
            for p in roster:
                people_left = roster[:]
                people_left.remove(p)
                newtotal = total+p
                if newtotal > halftotalscore: continue
                newmoves[newtotal] = people_left
        oldmoves = newmoves

    solution = min(map(lambda i: (abs(float(i)-halftotalscore), i), oldmoves.keys()))
    return (solution[1], sum(oldmoves[solution[1]]), oldmoves[solution[1]])

In [ ]:
print(ns[:100])
sol = team(ns[:100])
sol

In [ ]:
def team(t):
    sum_it = lambda a: sum[el[1] for el in a]
    
    iterations = range(2, len(t)/2+1)

    totalscore = sum_it(t)
    halftotalscore = totalscore/2.0

    oldmoves = {}

    for p in t:
        people_left = t[:]
        people_left.remove(p)
        oldmoves[p] = people_left

    if iterations == []:
        solution = min(map(lambda i: (abs(float(i)-halftotalscore), i), oldmoves.keys()))
        return (solution[1], sum(oldmoves[solution[1]]), oldmoves[solution[1]])

    for n in iterations:
        newmoves = {}
        for total, roster in oldmoves.iteritems():
            for p in roster:
                people_left = roster[:]
                people_left.remove(p)
                newtotal = total+p
                if newtotal > halftotalscore: continue
                newmoves[newtotal] = people_left
        oldmoves = newmoves

    solution = min(map(lambda i: (abs(float(i)-halftotalscore), i), oldmoves.keys()))
    return (solution[1], sum(oldmoves[solution[1]]), oldmoves[solution[1]])

In [ ]:
// Input:
// Values (stored in array v)
// Weights (stored in array w)
// Number of distinct items (n)
// Knapsack capacity (W)
// NOTE: The array "v" and array "w" are assumed to store all relevant values starting at index 1.

for j from 0 to W do:
    m[0, j] := 0

for i from 1 to n do:
    for j from 0 to W do:
        if w[i] > j then:
            m[i, j] := m[i-1, j]
        else:
            m[i, j] := max(m[i-1, j], m[i-1, j-w[i]] + v[i])

In [ ]:
V = ns
W = [1 for _ in V]
n = train_df.shape[0]

In [ ]:
/**
 * Returns the indices of the items of the optimal knapsack.
 * i: We can include items 1 through i in the knapsack
 * j: maximum weight of the knapsack
 */
function knapsack(i: int, j: int): Set<int> {
    if i == 0 then:
        return {}
    if m[i, j] > m[i-1, j] then:
        return {i} ∪ knapsack(i-1, j-w[i])
    else:
        return knapsack(i-1, j)
}

knapsack(n, W)

In [ ]:
# Python program to demonstrate working of Meet in the
# Middle algorithm for maximum subset sum problem.
from typing import List
import bisect
X = [0] * 2000005
Y = [0] * 2000005

# Find all possible sum of elements of a[] and store
# in x[]
def calcsubarray(a: List[int], x: List[int], n: int, c: int) -> None:
	for i in range((1 << n)):
		s = 0
		for j in range(n):
			if (i & (1 << j)):
				s += a[j + c]
		x[i] = s

# Returns the maximum possible sum less or equal to S
def solveSubsetSum(a: List[int], n: int, S: int) -> int:
	global Y
	
	# Compute all subset sums of first and second
	# halves
	calcsubarray(a, X, n // 2, 0)
	calcsubarray(a, Y, n - n // 2, n // 2)
	size_X = 1 << (n // 2)
	size_Y = 1 << (n - n // 2)

	# Sort Y (we need to do doing binary search in it)
	YY = Y[:size_Y]
	YY.sort()
	Y = YY

	# To keep track of the maximum sum of a subset
	# such that the maximum sum is less than S
	maxx = 0

	# Traverse all elements of X and do Binary Search
	# for a pair in Y with maximum sum less than S.
	for i in range(size_X):

		if (X[i] <= S):

			# lower_bound() returns the first address
			# which has value greater than or equal to
			# S-X[i].
			p = bisect.bisect_left(Y, S - X[i])

			# If S-X[i] was not in array Y then decrease
			# p by 1
			if (p == size_Y or (p < size_Y and Y[p] != (S - X[i]))):
				p -= 1
			if ((Y[p] + X[i]) > maxx):
				maxx = Y[p] + X[i]
	return maxx

# Driver code
if __name__ == "__main__":

	a = [3, 34, 4, 12, 5, 2]
	n = len(a)
	S = 10
	print("Largest value smaller than or equal to given sum is {}".format(
		solveSubsetSum(a, n, S)))

# This code is contributed by sanjeev2552


In [ ]:
print(train_df.shape, valid_df.shape, test_df.shape)
train_df = get_binary_df(label, train_df)
valid_df = get_binary_df(label, valid_df)
test_df = get_binary_df(label, test_df)

In [ ]:
# Compute label weights
train_label = train_df[[label]].values
neg_weights, pos_weights = compute_class_freqs(train_label)
neg_weights, pos_weights = torch.Tensor(neg_weights), torch.Tensor(pos_weights)
print(neg_weights, pos_weights)

# Get transforms
train_tfs, test_tfs = get_transforms(image_size=image_size)

# Create datasets
train_ds = CRX8_Data(train_df, get_image_path(), label, image_size=image_size, transforms=train_tfs)
valid_ds = CRX8_Data(valid_df, get_image_path(), label, image_size=image_size, transforms=test_tfs)
test_ds  = CRX8_Data(test_df , get_image_path(), label, image_size=image_size, transforms=test_tfs)

# Create dataloaders
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=bs, shuffle=False)
test_dl  = DataLoader(test_ds,  batch_size=bs, shuffle=False)
dataloaders = {
    "train": train_dl,
    "val": valid_dl,
    "test": test_dl
}